# Módulo 7 - Actividad 1:
# Clasificador de imágenes manuscrites con redes neuronales profundas

## Objetivo
Implementar una red neuronal completamente conectada para clasificar imágenes del dataset Fashion MNIST. El estudiante deberá explorar diferentes combinaciones de capas, funciones de activación, funciones de pérdida y optimizadores, evaluar su rendimiento y reflexionar sobre los resultados.

**Datasets utilizados:**  
`Fashion MNIST`

---

### Estructura del Notebook:
1. Metodología.
2. Configuración del entorno.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

### Flujo de trabajo

1. **Carga, escalado y One-Hot Encoding:**
    - Se carga el dataset **Fashion MNIST** y se escala y se realiza One-Hot Encoding.

2. **Creación de modelo y aplicación del mismo con diferentes configuraciones:**
    - Define una función para la creación de una red neuronal secuencial con 2 capas ocultas y softmax en la salida.
    - Define una función para entrenar el modelo anterior.
    - Entrena el modelo en base a diferentes configuraciones de funciones de pérdida y optimizadores:
        - categorical_crossentropy + adam
        - categorical_crossentropy + sgd
        - mse + adam
        - mse + sgd

3. **Visualización e interpretación:**
    - Tabla resumen con la función de perdida y optimizador usado, además de test accuracy y test loss.

---

# 2. Configuración del entorno

--- 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 1:** Carga y preprocesamiento de datos.

- **`cargar_datos()`** 
Carga el dataset Fashion MNIST, lo normaliza y aplica one-hot encoding a las etiquetas.

---

##### `Decisiones de diseño`

##### **Escalado y One-Hot Encoding:**

- Las imágenes del dataset Fashion MNIST contienen valores de píxeles que van de 0 a 255. Escalar estos valores a un rango entre 0 y 1 (dividiendo por 255) ayuda a que el modelo entrene más rápido y de manera más estable, ya que redes neuronales suelen converger mejor cuando las entradas están normalizadas. Además, reduce problemas numéricos relacionados con valores muy grandes y ayuda a que las funciones de activación como ReLU o sigmoid funcionen dentro de rangos óptimos.

- Las etiquetas originales son enteros que indican la clase (0 a 9). Convertirlas a formato one-hot (vectores binarios donde solo la posición de la clase correcta es 1 y el resto 0) es necesario para tareas de clasificación con redes neuronales, especialmente cuando se usa una función de pérdida como categorical_crossentropy. Este formato permite que el modelo aprenda a predecir la probabilidad de cada clase y facilita el cálculo del error durante el entrenamiento.

---

In [ ]:
def cargar_datos():
    """
    Carga y preprocesa el dataset Fashion MNIST.

    - Normaliza los valores de los píxeles a un rango entre 0 y 1.
    - Convierte las etiquetas a codificación one-hot.

    Returns:
        tuple: (x_train, y_train_oh, x_test, y_test_oh)
            - x_train (ndarray): Imágenes de entrenamiento normalizadas.
            - y_train_oh (ndarray): Etiquetas de entrenamiento en formato one-hot.
            - x_test (ndarray): Imágenes de prueba normalizadas.
            - y_test_oh (ndarray): Etiquetas de prueba en formato one-hot.
    """
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train_oh = to_categorical(y_train, 10)
    y_test_oh = to_categorical(y_test, 10)
    return x_train, y_train_oh, x_test, y_test_oh

**Bloque 2:** Definición y entrenamiento de modelos.

- **`build_model()`** 
Construye una red neuronal secuencial con dos capas ocultas y salida softmax para clasificación multiclase.

- **`train_model()`** 
Compila y entrena el modelo usando los datos de entrenamiento y validación con los parámetros indicados.

- **`entrenamientos()`** 
Ejecuta entrenamientos con distintas combinaciones de funciones de pérdida y optimizadores, registrando resultados.

---

##### `Decisiones de diseño`

##### **Eleccion de multiples funciones sobre una sola que cree y entrene el modelo:**

- Dividir el código en funciones pequeñas y específicas mejora la claridad, facilita el mantenimiento y permite reutilizar partes del código fácilmente. Al usar build_model dentro de train_model y esta dentro de entrenamientos, se mantiene una estructura modular que simplifica probar distintas configuraciones, hacer cambios puntuales y escalar el proyecto sin que el código se vuelva confuso o difícil de manejar.

---

In [ ]:
def build_model(activation1='relu', activation2='tanh'):
    """
    Construye una red neuronal secuencial con dos capas ocultas y softmax en la salida.

    Args:
        activation1 (str): Función de activación de la primera capa oculta. Default 'relu'.
        activation2 (str): Función de activación de la segunda capa oculta. Default 'tanh'.

    Returns:
        keras.Model: Modelo compilado sin entrenar.
    """
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(128, activation=activation1))
    model.add(Dense(64, activation=activation2))
    model.add(Dense(10, activation='softmax'))
    return model

def train_model(x_train, y_train_oh, loss, optimizer, epochs=10):
    """
    Compila y entrena un modelo con una función de pérdida y optimizador dados.

    Usa los datos globales `x_train` y `y_train_oh`.

    Args:
        x_train (ndarray): Datos de entrenamiento normalizados.
        y_train_oh (ndarray): Etiquetas de entrenamiento en formato one-hot.
        loss (str): Nombre de la función de pérdida ('categorical_crossentropy' o 'mse').
        optimizer (str): Optimizador a usar ('adam', 'sgd', etc.).
        epochs (int): Número de épocas para entrenar. Default 10.

    Returns:
        tuple: (modelo entrenado, historial de entrenamiento)
    """
    model = build_model()
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    history = model.fit(
        x_train, y_train_oh,
        validation_split=0.2,
        epochs=epochs,
        batch_size=128,
        verbose=0
    )
    return model, history

def entrenamientos(x_train, y_train_oh, x_test, y_test_oh, epochs=10):
    """
    Entrena el modelo con distintas combinaciones de funciones de pérdida y optimizadores.

    Evalúa cada modelo en el conjunto de prueba.

    Args:
        x_train (ndarray): Datos de entrenamiento normalizados.
        y_train_oh (ndarray): Etiquetas de entrenamiento en formato one-hot.
        x_test (ndarray): Datos de prueba normalizados.
        y_test_oh (ndarray): Etiquetas de prueba en formato one-hot.
        epochs (int): Número de épocas para cada entrenamiento. Default 10.

    Returns:
        tuple: (resultados, historial)
            - resultados (list): Lista de dicts con métricas de evaluación.
            - historias (list): Lista de tuplas (loss, opt, history).
    """
    resultados = []
    historial = []

    combinaciones = [
        ('categorical_crossentropy', 'adam'),
        ('categorical_crossentropy', 'sgd'),
        ('mse', 'adam'),
        ('mse', 'sgd'),
    ]

    for loss, opt in combinaciones:
        print(f"--- Entrenando con loss={loss}, optimizer={opt} ---")
        model, history = train_model(x_test, y_test_oh, loss, opt, epochs=epochs)

        test_loss, test_acc = model.evaluate(x_test, y_test_oh, verbose=0)
        resultados.append({
            'Loss Function': loss,
            'Optimizer': opt,
            'Test Accuracy': test_acc,
            'Test Loss': test_loss
        })

        historial.append((loss, opt, history))

    return resultados, historial

**Bloque 3:** Visualizaciones.

- **`mostrar_resultados()`** 
Muestra una tabla comparativa con la precisión y pérdida del conjunto de prueba para cada experimento.

- **`graficar_historias()`** 
Genera gráficos de las curvas de pérdida y precisión de entrenamiento y validación para cada combinación evaluada.

---

##### `Decisiones de diseño`

##### **Justificación para graficar las curvas de pérdida y precisión:**

- Visualizar las curvas de pérdida y precisión durante el entrenamiento permite entender cómo evoluciona el aprendizaje del modelo en cada época, identificar posibles problemas como sobreajuste o subajuste, y comparar el desempeño entre diferentes configuraciones. Estas gráficas facilitan interpretar la estabilidad y eficacia del entrenamiento, ayudando a tomar decisiones informadas para mejorar el modelo.

---

In [ ]:
def mostrar_resultados(resultados):
    """
    Muestra una tabla con los resultados de los experimentos.

    Args:
        resultados (list): Lista de diccionarios con métricas de evaluación.

    Returns:
        pandas.DataFrame: Tabla de resultados mostrada y devuelta.
    """
    df_resultados = pd.DataFrame(resultados)
    print("\n===== Comparación de resultados en test =====")
    df_resultados.to_string(index=False)
    display(df_resultados)
    return df_resultados

def graficar_historias(historial):
    """
    Grafica curvas de pérdida y precisión para cada combinación entrenada.

    Args:
        historias (list): Lista de tuplas (loss, opt, history) de cada experimento.
    """
    plt.figure(figsize=(14, 10))

    for i, (loss, opt, history) in enumerate(historial):
        plt.subplot(2, 2, i + 1)
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title(f'{opt.capitalize()} con {loss.capitalize()}')
        #plt.title(f'{loss} + {opt}')
        plt.xlabel('Epoch')
        plt.ylabel('Metric')
        plt.legend()
        plt.grid(True)

    plt.tight_layout()
    plt.show()

**Bloque 4:** Función de ejecución.

- **`main()`** 
Ejecuta el pipeline completo de carga, entrenamiento y visualización de resultados.

---

In [ ]:
def main():
    """
    Punto de entrada del programa:
    - Carga y prepara los datos.
    - Ejecuta los entrenamientos con combinaciones de pérdida/optimizador.
    - Muestra resultados en tabla y gráficos.
    """
    # Carga de datos
    x_train, y_train_oh, x_test, y_test_oh = cargar_datos()

    # Entrenamiento de modelos con diferentes configuraciones
    resultados, historial = entrenamientos(x_train, y_train_oh, x_test, y_test_oh,epochs=10)

    # Visualización de los resultados comparativos en tablas y gráficas
    mostrar_resultados(resultados)
    graficar_historias(historial)

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == '__main__':
    main()

# 5. Análisis de los resultados y reflexiones finales

---

## Análisis comparativo de una red neuronal con diferentes configuraciones

> Nota: el análisis se realizo en base a una ejecución en concreto, por lo que los valores podrían cambiar entre ejecuciones, sin embargo, la base no cambia, solo varían un poco los valores pero siguen siendo igual de representativos para la evaluación.

#### 1. ¿Qué combinación funcionó mejor y por qué?

- La combinación que obtuvo mejor desempeño fue categorical_crossentropy con el optimizador adam, alcanzando un accuracy en test cercano al 88.6%. Este resultado se explica porque la función de pérdida categorical_crossentropy es la más adecuada para problemas de clasificación multiclase, al penalizar de manera efectiva las predicciones incorrectas y guiar mejor el aprendizaje. Por otro lado, el optimizador adam ajusta dinámicamente la tasa de aprendizaje para cada parámetro, acelerando la convergencia y mejorando la estabilidad durante el entrenamiento. Las curvas de pérdida y precisión muestran una mejora consistente y un buen balance entre entrenamiento y validación, indicando que el modelo generaliza bien sin sobreajustar.

---

#### 2. ¿Qué efectos tuviste al cambiar funciones de activación o pérdida?

- En cuanto a las funciones de pérdida, se observa que aunque el mse logró un accuracy alto con adam (~88.2%), su valor numérico de pérdida fue inusualmente bajo, lo que puede ser engañoso y no refleja adecuadamente la calidad del modelo para clasificación. Además, cuando se usó sgd con mse, el modelo no aprendió correctamente, con un accuracy muy bajo (17.3%), indicando que esta combinación no es adecuada para esta tarea. En contraste, la función categorical_crossentropy mostró una convergencia más rápida y mejores resultados con ambos optimizadores, especialmente con adam.

- Respecto a las funciones de activación, se mantuvieron constantes: ReLU en la primera capa y Tanh en la segunda. Esta combinación equilibra la eficiencia del aprendizaje inicial (gracias a ReLU) con la capacidad de modelar relaciones no lineales más suaves (gracias a Tanh). Dado que no se variaron estas activaciones entre experimentos, el cambio en rendimiento se atribuye principalmente a la función de pérdida y el optimizador.

---

#### 3. ¿Qué harías diferente si tuvieras más datos o más tiempo de entrenamiento?

- Con más datos disponibles, se podría considerar aumentar la complejidad del modelo, añadiendo más capas o neuronas, junto con técnicas de regularización como Dropout o Batch Normalization para evitar sobreajuste. Esto aprovecharía mejor la riqueza de los datos adicionales para mejorar la generalización.

- Si se dispusiera de más tiempo para entrenar, sería recomendable extender el número de épocas (por ejemplo, a 20 o 30) y utilizar técnicas como EarlyStopping para evitar el sobreentrenamiento. También valdría la pena experimentar con el tamaño del batch para optimizar el proceso de entrenamiento y aplicar esquemas de ajuste dinámico de la tasa de aprendizaje (learning rate schedules), especialmente cuando se usa SGD.

- Finalmente, para optimizar hiperparámetros, herramientas automáticas como Optuna o Grid Search podrían explorar distintas configuraciones, incluyendo la cantidad de neuronas, funciones de activación alternativas (LeakyReLU, ELU), y parámetros del optimizador (learning rate, momentum), buscando el mejor equilibrio entre precisión y eficiencia.

---

### Conclusión

La combinación categorical_crossentropy + adam resultó ser la más efectiva, proporcionando el mejor balance entre precisión y estabilidad en el entrenamiento. Aunque mse mostró resultados competitivos con adam, no es la función de pérdida recomendada para clasificación multiclase, y su desempeño fue pobre con SGD. Para futuros trabajos, se sugiere aumentar la capacidad del modelo y el tiempo de entrenamiento, aplicando técnicas de regularización y ajuste fino de hiperparámetros para maximizar el rendimiento y la robustez del modelo.

---